In [4]:
import pandas as pd
import numpy as np
import operator

In [5]:
mdf = '/Users/siero5335/Desktop/InstaCart/'
print('loading prior orders')
prior_orders = pd.read_csv(mdf + 'order_products__prior.csv', dtype={
        'order_id': np.int32,
        'product_id': np.int32,
        'add_to_cart_order': np.int16,
        'reordered': np.int8})
print('loading orders')
orders = pd.read_csv(mdf + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})
print('loading aisles info')
aisles = pd.read_csv(mdf + 'products.csv', engine='c',
                           usecols = ['product_id','aisle_id'],
                       dtype={'product_id': np.int32, 'aisle_id': np.int32})
pd.set_option('display.float_format', lambda x: '%.3f' % x)

prior_orders.shape
orders.shape

loading prior orders
loading orders
loading aisles info


(3421083, 7)

In [6]:
# removing all user_ids not in the test set from both files to save memory
# the test users present ample data to make models. (and saves space)
test  = orders
user_ids = test['user_id'].values
order_ids = test['order_id'].values
orders = orders[orders['user_id'].isin(user_ids)]

#del test
test.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,nan
1,2398795,1,prior,2,3,7,15.000
2,473747,1,prior,3,3,12,21.000
3,2254736,1,prior,4,4,7,29.000
4,431534,1,prior,5,4,15,28.000


In [7]:
# Calculate the Prior : p(reordered|product_id)
prior = pd.DataFrame(prior_orders.groupby('product_id')['reordered'].agg([('number_of_orders',len),
        ('sum_of_reorders','sum')]))
prior['prior_p'] = (prior['sum_of_reorders']+1)/(prior['number_of_orders']+2)
del prior['number_of_orders']
del prior['sum_of_reorders']
print('Here is The Prior: our first guess of how probable it is that a product be reordered.')
#prior.to_csv('prior_out.csv')

prior.head(3)

Here is The Prior: our first guess of how probable it is that a product be reordered.


,prior_p
product_id,
1,0.613
2,0.141
3,0.731


In [8]:
# merge everything into one dataframe and save any memory space

comb = pd.DataFrame()
comb = pd.merge(prior_orders, orders, on='order_id', how='right')
# slim down comb - 
del comb['eval_set']
del comb['order_dow']
del comb['order_hour_of_day']
del prior_orders
del orders

comb = pd.merge(comb, aisles, on ='product_id', how = 'left')
del aisles

prior.reset_index(inplace = True)
comb = pd.merge(comb, prior, on ='product_id', how = 'left')
del prior
print('combined data in DataFrame comb')
comb.head(5)

combined data in DataFrame comb


,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order,aisle_id,prior_p
0,2,33120.000,1.000,1.000,202279,3,8.000,86.000,0.708
1,2,28985.000,2.000,1.000,202279,3,8.000,83.000,0.694
2,2,9327.000,3.000,0.000,202279,3,8.000,104.000,0.158
3,2,45918.000,4.000,1.000,202279,3,8.000,19.000,0.273
4,2,30035.000,5.000,0.000,202279,3,8.000,17.000,0.508


In [9]:
# Build the factors needed for a model of probability of reordered. This model forms our
# hypothesis H and allows the calculation of each Bayes Factor: BF = p(D|H)/(1-p(D|H))
# where D is the test user product buying history. See DAG of model above.

# discretize reorder count into categories, 9 buckets, being sure to include 0 as bucket
# These bins maximize mutual information with ['reordered']. Done outside python
recount = pd.DataFrame()
recount['reorder_c'] = comb.groupby(comb.order_id)['reordered'].sum().fillna(0)
bins = [-0.1, 0, 2,4,6,8,11,14,19,71]
cat =  ['None','<=2','<=4','<=6','<=8','<=11','<=14','<=19','>19']
recount['reorder_b'] = pd.cut(recount['reorder_c'], bins, labels = cat)
recount.reset_index(inplace = True)
comb = pd.merge(comb, recount, how = 'left', on = 'order_id')
del recount

# discretize 'add_to_cart_order' (atco) into categories, 8 buckets
# These bins maximize mutual information with ['recount']. Done outside python
bins = [0,2,3,5,7,9,12,17,80]
cat = ['<=2','<=3','<=5','<=7','<=9','<=12','<=17','>17']
comb['atco1'] = pd.cut(comb['add_to_cart_order'], bins, labels = cat)
del comb['add_to_cart_order']
print('comb ')
comb.head(2)

comb 


,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,aisle_id,prior_p,reorder_c,reorder_b,atco1
0,2,33120.000,1.000,202279,3,8.000,86.000,0.708,6.000,<=6,<=2
1,2,28985.000,1.000,202279,3,8.000,83.000,0.694,6.000,<=6,<=2


In [10]:
# these are the children Nodes of reordered:atco, aisle, recount. Build occurrence tables
# first, then calculate probabilities. Then merge to add atco into comb.
# 
atco_fac = pd.DataFrame()
atco_fac = comb.groupby(['reordered', 'atco1'])['atco1'].agg(np.count_nonzero).unstack('atco1')
tot = pd.DataFrame()
tot = np.sum(atco_fac,axis=1)
atco_fac = atco_fac.iloc[:,:].div(tot, axis=0)
atco_fac = atco_fac.stack('atco1')
atco_fac = pd.DataFrame(atco_fac)
atco_fac.reset_index(inplace = True)
atco_fac.rename(columns = {0:'atco_fac_p'}, inplace = True)
comb = pd.merge(comb, atco_fac, how='left', on=('reordered', 'atco1'))

comb.head(2)

,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,aisle_id,prior_p,reorder_c,reorder_b,atco1,atco_fac_p
0,2,33120.000,1.000,202279,3,8.000,86.000,0.708,6.000,<=6,<=2,0.222
1,2,28985.000,1.000,202279,3,8.000,83.000,0.694,6.000,<=6,<=2,0.222


In [11]:
# calculate other two factors' probability tables, then probability
# and merge into comb

aisle_fac = pd.DataFrame()
aisle_fac = comb.groupby(['reordered', 'atco1', 'aisle_id'])['aisle_id']\
                .agg(np.count_nonzero).unstack('aisle_id')
tot = np.sum(aisle_fac,axis=1)
aisle_fac = aisle_fac.iloc[:,:].div(tot, axis=0)
aisle_fac = aisle_fac.stack('aisle_id')
aisle_fac = pd.DataFrame(aisle_fac)
aisle_fac.reset_index(inplace = True)
aisle_fac.rename(columns = {0:'aisle_fac_p'}, inplace = True)
comb = pd.merge(comb, aisle_fac, how = 'left', on = ('aisle_id','reordered','atco1'))

comb.head(2)

,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,aisle_id,prior_p,reorder_c,reorder_b,atco1,atco_fac_p,aisle_fac_p
0,2,33120.000,1.000,202279,3,8.000,86.000,0.708,6.000,<=6,<=2,0.222,0.021
1,2,28985.000,1.000,202279,3,8.000,83.000,0.694,6.000,<=6,<=2,0.222,0.076


In [12]:
# last factor is reorder_count_factor   
    
recount_fac = pd.DataFrame()
recount_fac = comb.groupby(['reordered', 'atco1', 'reorder_b'])['reorder_b']\
                    .agg(np.count_nonzero).unstack('reorder_b')

tot = pd.DataFrame()
tot = np.sum(recount_fac,axis=1)
recount_fac = recount_fac.iloc[:,:].div(tot, axis=0)
recount_fac.stack('reorder_b')
recount_fac = pd.DataFrame(recount_fac.unstack('reordered').unstack('atco1')).reset_index()
recount_fac.rename(columns = {0:'recount_fac_p'}, inplace = True)
comb = pd.merge(comb, recount_fac, how = 'left', on = ('reorder_b', 'reordered', 'atco1'))

recount_fac.head(3)

,reorder_b,reordered,atco1,recount_fac_p
0,<=11,0.000,<=12,0.180
1,<=11,0.000,<=17,0.200
2,<=11,0.000,<=2,0.044


In [13]:
# slim down combined DataFrame to save space and speed up processing
del comb['atco1']
del comb['reorder_b']
del comb['aisle_id']
#del comb['order_number']
comb.head(5)

,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,prior_p,reorder_c,atco_fac_p,aisle_fac_p,recount_fac_p
0,2,33120.000,1.000,202279,3,8.000,0.708,6.000,0.222,0.021,0.175
1,2,28985.000,1.000,202279,3,8.000,0.694,6.000,0.222,0.076,0.175
2,2,9327.000,0.000,202279,3,8.000,0.158,6.000,0.074,0.013,0.090
3,2,45918.000,1.000,202279,3,8.000,0.273,6.000,0.168,0.004,0.217
4,2,30035.000,0.000,202279,3,8.000,0.508,6.000,0.143,0.017,0.117


In [14]:
# Use the factors in comb + the prior_p to update a posterior for each product purchased.
# work in progress on beta
# Use a test beta = 95% per month for memory retention function of users. Akin to Recency.

comb['p'] = (comb['atco_fac_p'] * comb['aisle_fac_p'] * comb['recount_fac_p'])

#split into two dataframes on reordered == 1 and == 0

comb0 = pd.DataFrame()
comb0 = comb[comb['reordered']==0]
# now every product that was ordered has a posterior in usr.
comb0['beta'] = 1
comb0['bf'] = comb0['prior_p'] * comb0['p']/(1 - comb0['p'])
# Small 'slight of hand' here. comb0.bf is really the first posterior and second prior.

# Calculate beta and BF1 for the reordered products
comb1 = pd.DataFrame()
comb1 = comb[comb['reordered']==1]
comb1['beta'] = 1 - .05*comb1['days_since_prior_order']/30
comb1['bf'] = (1 - comb1['p'])/comb1['p']

comb_last = pd.DataFrame()
comb_last = pd.concat([comb0, comb1], axis=0).reset_index(drop=True)

comb_last.head(5)

/Users/siero5335/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/siero5335/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/siero5335/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,prior_p,reorder_c,atco_fac_p,aisle_fac_p,recount_fac_p,p,beta,bf
0,2,9327.000,0.000,202279,3,8.000,0.158,6.000,0.074,0.013,0.090,0.000,1.000,0.000
1,2,30035.000,0.000,202279,3,8.000,0.508,6.000,0.143,0.017,0.117,0.000,1.000,0.000
2,2,43668.000,0.000,202279,3,8.000,0.248,6.000,0.108,0.051,0.191,0.001,1.000,0.000
3,4,46842.000,0.000,178520,36,7.000,0.672,12.000,0.152,0.006,0.024,0.000,1.000,0.000
4,5,47209.000,0.000,156122,42,9.000,0.797,21.000,0.136,0.059,0.230,0.002,1.000,0.001


In [15]:
comb_l = pd.DataFrame()
comb_l['reordered'] = comb_last['reordered']
comb_l['user_id'] = comb_last['user_id']
comb_l['product_id'] = comb_last['product_id']
comb_l['bf'] = comb_last['bf'].astype(np.float64)
comb_l['beta'] = comb_last['beta'].astype(np.float32)
comb_l['recount_c']= comb_last['reorder_c']
comb_l['order_number']= comb_last['order_number']
del comb_last
comb_l = comb_l.sort_values((['user_id', 'order_number', 'bf']))
pd.set_option('display.float_format', lambda x: '%.6f' % x)
comb_l.head(5)

,reordered,user_id,product_id,bf,beta,recount_c,order_number
9875840,0.000000,1,12427.000000,0.000082,1.000000,0.000000,1
9875841,0.000000,1,26088.000000,0.000098,1.000000,0.000000,1
9875842,0.000000,1,26405.000000,0.000149,1.000000,0.000000,1
9875838,0.000000,1,196.000000,0.000631,1.000000,0.000000,1
9875839,0.000000,1,14084.000000,0.000911,1.000000,0.000000,1


In [16]:
# finally, perform update of every product 

usr = pd.DataFrame()
# Find way to introduce beta to the update. Also, the update
# must not occur until reordered==0 (aka: when first ordered)
first_order = pd.DataFrame()
first_order = comb_l[comb_l.reordered==0]
first_order.rename(columns = {'order_number':'first_o'}, inplace = True)
first_ords = pd.DataFrame()
first_ords['user_id'] = first_order['user_id']
first_ords['product_id'] = first_order['product_id']
first_ords['first_o'] = first_order['first_o']
us = pd.DataFrame()
us = pd.merge(comb_l, first_ords, on = ('user_id', 'product_id'), how = 'left')



us.head(5)

/Users/siero5335/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,reordered,user_id,product_id,bf,beta,recount_c,order_number,first_o
0,0.000000,1,12427.000000,0.000082,1.000000,0.000000,1,1
1,0.000000,1,26088.000000,0.000098,1.000000,0.000000,1,1
2,0.000000,1,26405.000000,0.000149,1.000000,0.000000,1,1
3,0.000000,1,196.000000,0.000631,1.000000,0.000000,1,1
4,0.000000,1,14084.000000,0.000911,1.000000,0.000000,1,1


In [17]:
update = lambda x, y: x**y;

usr = us[us.order_number > us.first_o].groupby(['user_id','product_id'])['bf']\
    .agg(np.prod).reset_index()

temp = pd.DataFrame()
temp = us.groupby(['user_id'])['recount_c'].agg(np.mean).reset_index()
user = pd.merge(usr, temp, on = 'user_id', how = 'left')

user.sort_values(((['bf', 'user_id'])), ascending=False)

pd.set_option('display.float_format', lambda x: '%.1f' % x)
user.head(20)

,user_id,product_id,bf,recount_c
0,1,196.0,10702590211448937080434982912.0,4.3
1,1,10258.0,791184262213878093107127386112.0,4.3
2,1,12427.0,44692992519339118205344214518071296.0,4.3
3,1,13032.0,26197639.8,4.3
4,1,13176.0,998.6,4.3
5,1,25133.0,3152157497306953809920.0,4.3
6,1,26088.0,8653.9,4.3
7,1,26405.0,4130.0,4.3
8,1,46149.0,1754767.6,4.3
9,1,49235.0,1677.4,4.3


In [18]:
# Credit for this code belongs to Kareem Eissa. You are very generous and skilled!

def f1(x):
    return ' '.join([str(int(a)) for a in x])
def f2(x):
    return 'None'

u = pd.DataFrame()
uu = pd.DataFrame()
uuu = pd.DataFrame()

u = user.reset_index().sort_values(((['user_id','bf'])), ascending=False)
u['cumulative'] = u.groupby('user_id').cumcount()
u = pd.merge(test, u, on='user_id')[['order_id', 'product_id', 'recount_c', 'cumulative', 'user_id', 'bf']].sort_values('order_id')

# these orders include all orders with expected reordered products
uu = u[(u.recount_c >= u.cumulative + 1)].groupby('order_id').agg({'product_id': f1})
# these orders have expected "None" reordered products or recount_c < 0.5
uuu = u[(u.recount_c < 0.5)].groupby('order_id').agg({'product_id': f2})

subm = pd.DataFrame()
subm = pd.concat([uu, uuu], keys = 'order_id', axis=0).reset_index()
subm.sort_values((('order_id')), ascending=False)

subm[['order_id', 'product_id']].to_csv('bayesian.csv', index=False)
u.head(10)

,order_id,product_id,recount_c,cumulative,user_id,bf
89001438,1,22035.0,2.9,5,112108,932.7
89001433,1,14947.0,2.9,0,112108,547535.9
89001434,1,30881.0,2.9,1,112108,26229.6
89001435,1,43633.0,2.9,2,112108,26229.6
89001436,1,11109.0,2.9,3,112108,3559.2
89001437,1,5707.0,2.9,4,112108,2036.2
89001439,1,49302.0,2.9,6,112108,504.5
89001440,1,44359.0,2.9,7,112108,261.6
159163670,2,10960.0,5.7,6,202279,875903137.2
159163671,2,30035.0,5.7,7,202279,51906796.9


In [19]:
subm.shape

(3315747, 3)

In [20]:
u.shape

(162231165, 6)

In [21]:
u.to_csv("bf.csv")